In [158]:
# from flask import Flask, render_template, redirect
from flask import Flask, jsonify
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from flask_sqlalchemy import SQLAlchemy
import pandas as pd

In [2]:
engine = create_engine("sqlite:///Data/belly_button_biodiversity.sqlite")
inspector = inspect(engine)

**@app.route('/names')**

In [4]:
def find_col_names(eng, table_name):
    inspector = inspect(eng)
    tables = {}
    
    for table in inspector.get_table_names():
        columns = inspector.get_columns(table)
        names = []
        for c in columns:
            names.append(c['name'])
            tables[table] = names
    
    column_names = tables[table_name]        
    del column_names[:1] #del first name(index)
    
    return column_names

In [7]:
tables = {}
    
for table in inspector.get_table_names():
    columns = inspector.get_columns(table)
    names = []
    for c in columns:
        names.append(c['name'])
        tables[table] = names

In [9]:
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [10]:
Samples_metadata = Base.classes.samples_metadata
Samples = Base.classes.samples
Otu = Base.classes.otu

**@app.route('/otu')**

In [32]:
bact_tuples = session.query(Otu.lowest_taxonomic_unit_found).all()
bacteries = []

In [37]:
for b_tuple in bact_tuples:
    for bact in b_tuple:
        bacteries.append(bact)
#bacteries = [bact for b_tuple in bact_tuples for bact in b_tuple]

**@app.route('/metadata/<sample>')**

In [130]:
def metasample(eng, sample):
    if sample in find_col_names(eng, 'samples'):
        form_sample = int(sample[3:])
    
        keys = ['AGE', 'BBTYPE', 'ETHNICITY', 'GENDER', 'LOCATION', 'SAMPLEID']
        values = [f for s in eng.execute(f'SELECT * FROM samples_metadata where SAMPLEID = {form_sample}').\
          fetchall() for f in s]
    
        sample_dict = {}
        cols = inspector.get_columns('samples_metadata')
        for x in range(len(cols)):
            if cols[x]['name'] in keys:
                sample_dict[cols[x]['name']] = values[x]
        return sample_dict
    else:
        return f'There is no sample named {sample} in our dataset'

In [131]:
metasample(engine, 'BB_1415')

{'AGE': 0,
 'BBTYPE': 'Unknown',
 'ETHNICITY': '0',
 'GENDER': '0',
 'LOCATION': '0',
 'SAMPLEID': 1415}

In [78]:
keys = ['AGE', 'BBTYPE', 'ETHNICITY', 'GENDER', 'LOCATION', 'SAMPLEID']

In [125]:
q = 'BB_1265'

In [75]:
r = session.query(Samples_metadata.AGE, Samples_metadata.BBTYPE, Samples_metadata.ETHNICITY,\
                 Samples_metadata.GENDER, Samples_metadata.LOCATION, Samples_metadata.SAMPLEID).\
                filter(Samples_metadata.SAMPLEID == q).all()

In [66]:
values = [y for x in r for y in x]

In [76]:
sample_dict = {}
for x in range(len(keys)):
    sample_dict[keys[x]] = values[x]

In [127]:
metasample(engine, 'BB_1286')

{'AGE': 38,
 'BBTYPE': 'I',
 'ETHNICITY': 'Caucasian',
 'GENDER': 'M',
 'LOCATION': '0',
 'SAMPLEID': 1286}

In [77]:
sample_dict

{'AGE': 44,
 'BBTYPE': 'I',
 'ETHNICITY': 'European',
 'GENDER': 'M',
 'LOCATION': 'NewHaven/CT',
 'SAMPLEID': 944}

In [87]:
values = [f for s in engine.execute(f'SELECT * FROM samples_metadata where SAMPLEID = {q}').\
          fetchall() for f in s]

In [95]:
sample_dict2 = {}
cols = inspector.get_columns('samples_metadata')
for x in range(len(cols)):
    if cols[x]['name'] in keys:
        sample_dict2[cols[x]['name']] = values[x]

In [ ]:
# labels, values = zip(*lyrics.items())  check it https://docs.python.org/3/library/functions.html#zip

In [96]:
sample_dict2

{'AGE': 44,
 'BBTYPE': 'I',
 'ETHNICITY': 'European',
 'GENDER': 'M',
 'LOCATION': 'NewHaven/CT',
 'SAMPLEID': 944}

In [148]:
t = 'BB_940'
int(t[3:])

940

910

In [126]:
if q in find_col_names(engine, 'samples'):
    print('yes')

yes


In [140]:
int(session.query(Samples_metadata.WFREQ).filter(Samples_metadata.SAMPLEID == 940).all()[0][0])

2

In [179]:
fetch = engine.execute(f'select "{t}", otu_id from samples where "{t}" > 0 order by "{t}" desc' ).fetchall()
val, ids = zip(*fetch)

In [183]:
values_otu = [dict(otu_ids=list(ids)), dict(sample_values=list(val))]
values_otu

[{'otu_ids': [1167,
   2859,
   482,
   2264,
   41,
   1189,
   352,
   189,
   1977,
   2318,
   3450,
   874,
   1959,
   2191,
   1950,
   2077,
   2275,
   944,
   2184,
   2244,
   2024,
   2419,
   2811,
   165,
   2782,
   2011,
   2247,
   2396,
   830,
   1795,
   2964,
   2722,
   307,
   2178,
   2908,
   1193,
   1208,
   2039,
   2167,
   1274,
   2737,
   2739,
   833,
   907,
   1314,
   1962,
   2186,
   2335,
   2936,
   121,
   159,
   170,
   258,
   259,
   340,
   342,
   357,
   412,
   513,
   725,
   1169,
   1232,
   1497,
   1498,
   1503,
   1505,
   1960,
   1968,
   2065,
   2110,
   2188,
   2235,
   2291,
   2342,
   2350,
   2475,
   2483,
   2491,
   2546,
   2571,
   11,
   19,
   22,
   24,
   39,
   42,
   79,
   81,
   84,
   93,
   102,
   122,
   133,
   154,
   185,
   198,
   226,
   235,
   244,
   246,
   300,
   314,
   320,
   327,
   328,
   359,
   361,
   373,
   381,
   386,
   400,
   404,
   407,
   414,
   564,
   593,
   620,
   650

In [169]:
pd.DataFrame([f for f in fetch], columns=['sample_values', 'otu_ids'])

,sample_values,otu_ids
0,163,1167
1,126,2859
2,113,482
3,78,2264
4,71,41
5,51,1189
6,50,352
7,47,189
8,40,1977
9,40,2318


In [111]:
find_col_names(engine, 'samples')

['BB_940',
 'BB_941',
 'BB_943',
 'BB_944',
 'BB_945',
 'BB_946',
 'BB_947',
 'BB_948',
 'BB_949',
 'BB_950',
 'BB_952',
 'BB_953',
 'BB_954',
 'BB_955',
 'BB_956',
 'BB_958',
 'BB_959',
 'BB_960',
 'BB_961',
 'BB_962',
 'BB_963',
 'BB_964',
 'BB_966',
 'BB_967',
 'BB_968',
 'BB_969',
 'BB_970',
 'BB_971',
 'BB_972',
 'BB_973',
 'BB_974',
 'BB_975',
 'BB_978',
 'BB_1233',
 'BB_1234',
 'BB_1235',
 'BB_1236',
 'BB_1237',
 'BB_1238',
 'BB_1242',
 'BB_1243',
 'BB_1246',
 'BB_1253',
 'BB_1254',
 'BB_1258',
 'BB_1259',
 'BB_1260',
 'BB_1264',
 'BB_1265',
 'BB_1273',
 'BB_1275',
 'BB_1276',
 'BB_1277',
 'BB_1278',
 'BB_1279',
 'BB_1280',
 'BB_1281',
 'BB_1282',
 'BB_1283',
 'BB_1284',
 'BB_1285',
 'BB_1286',
 'BB_1287',
 'BB_1288',
 'BB_1289',
 'BB_1290',
 'BB_1291',
 'BB_1292',
 'BB_1293',
 'BB_1294',
 'BB_1295',
 'BB_1296',
 'BB_1297',
 'BB_1298',
 'BB_1308',
 'BB_1309',
 'BB_1310',
 'BB_1374',
 'BB_1415',
 'BB_1439',
 'BB_1441',
 'BB_1443',
 'BB_1486',
 'BB_1487',
 'BB_1489',
 'BB_1490',
 